In [ ]:
!pip install finlab > log.txt
!pip install ta-lib-bin > log.txt
!pip install git+https://github.com/microsoft/qlib.git > log.txt

In [1]:
from finlab import data
from finlab.ml import feature as mlf
features = mlf.combine({
  'pb': data.get('price_earning_ratio:股價淨值比'),
  'pe': data.get('price_earning_ratio:本益比')
}, resample='W')

features.head()

pb     pe
datetime   instrument             
2010-01-10 1101        1.52  19.45
           1102        1.47  14.92
           1103        0.77  39.89
           1104        0.90  71.60
           1107         NaN    NaN

In [5]:
from finlab.ml import label as mll

label = mll.return_percentage(features.index, resample='W', period=12)

In [6]:
import finlab.ml.qlib as q

is_train = features.index.get_level_values('datetime') < '2020-01-01'

X_train, y_train = features[is_train], label[is_train]
X_test = features[~is_train]

model = q.XGBModel()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

[929:MainThread](2024-04-02 15:11:21,936) INFO - qlib.timer - [log.py:127] - Time cost: 0.554s | Loading data Done
INFO:qlib.timer:Time cost: 0.554s | Loading data Done
[929:MainThread](2024-04-02 15:11:22,010) INFO - qlib.timer - [log.py:127] - Time cost: 0.039s | DropnaLabel Done
INFO:qlib.timer:Time cost: 0.039s | DropnaLabel Done
[929:MainThread](2024-04-02 15:11:23,366) INFO - qlib.timer - [log.py:127] - Time cost: 1.345s | CSZScoreNorm Done
INFO:qlib.timer:Time cost: 1.345s | CSZScoreNorm Done
[929:MainThread](2024-04-02 15:11:23,372) INFO - qlib.timer - [log.py:127] - Time cost: 1.429s | fit & process data Done
INFO:qlib.timer:Time cost: 1.429s | fit & process data Done
[929:MainThread](2024-04-02 15:11:23,379) INFO - qlib.timer - [log.py:127] - Time cost: 1.997s | Init data Done
INFO:qlib.timer:Time cost: 1.997s | Init data Done
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:11:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimato

[0]	train-rmse:0.99957	valid-rmse:0.99967
[20]	train-rmse:0.99827	valid-rmse:0.99963
[40]	train-rmse:0.99762	valid-rmse:0.99988
[60]	train-rmse:0.99731	valid-rmse:1.00019


[929:MainThread](2024-04-02 15:11:30,023) INFO - qlib.timer - [log.py:127] - Time cost: 0.088s | Loading data Done
INFO:qlib.timer:Time cost: 0.088s | Loading data Done
[929:MainThread](2024-04-02 15:11:30,028) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | fit & process data Done
INFO:qlib.timer:Time cost: 0.000s | fit & process data Done
[929:MainThread](2024-04-02 15:11:30,033) INFO - qlib.timer - [log.py:127] - Time cost: 0.097s | Init data Done
INFO:qlib.timer:Time cost: 0.097s | Init data Done


In [12]:
from finlab.backtest import sim

close = data.get('price:收盤價')

position = y_pred[close > close.average(60)].is_largest(10)

sim(position, resample='Q')

<IPython.core.display.Javascript object>